解析氨基酸属性文件：

从给定的 AA531properties.txt 文件中解析氨基酸属性数据，构建一个包含531种理化属性的字典。文件的格式是每行一个氨基酸，由氨基酸名称和对应的属性值组成，以制表符分隔。忽略文件的头部行。
蛋白质氨基酸序列编码：

从给定的 ACEtriPeptidesSequencesActivities.txt 文件中解析蛋白质氨基酸序列和对应的活性值。文件的格式是每行包含氨基酸序列和相应的活性值，由制表符分隔。
使用解析得到的氨基酸属性字典，将氨基酸序列编码为特征矩阵。每个氨基酸对应531个属性，考虑序列长度为3的情况。
处理非规范氨基酸：

如果序列中的氨基酸不在氨基酸属性字典中，将其替换为全为零的531个属性。在替换时，输出警告信息。
保存结果：

将编码后的特征矩阵和活性值合并，并以制表符分隔的形式保存到名为 result.txt 的文件中。
提示：

#请使用 NumPy 库来处理数组和矩阵。
#考虑对非规范氨基酸的处理，确保程序能够正确处理各种输入情况。
#可以添加命令行参数来接收输入文件名、输出文件名以及其他必要的参数。

#读取氨基酸属性文件，转换为字典

<!-- 解析氨基酸属性文件：

从给定的 AA531properties.txt 文件中解析氨基酸属性数据，构建一个包含531种理化属性的字典。文件的格式是每行一个氨基酸，由氨基酸名称和对应的属性值组成，以制表符分隔。忽略文件的头部行。
蛋白质氨基酸序列编码：

从给定的 ACEtriPeptidesSequencesActivities.txt 文件中解析蛋白质氨基酸序列和对应的活性值。文件的格式是每行包含氨基酸序列和相应的活性值，由制表符分隔。
使用解析得到的氨基酸属性字典，将氨基酸序列编码为特征矩阵。每个氨基酸对应531个属性，考虑序列长度为3的情况。
处理非规范氨基酸：

如果序列中的氨基酸不在氨基酸属性字典中，将其替换为全为零的531个属性。在替换时，输出警告信息。
保存结果：

将编码后的特征矩阵和活性值合并，并以制表符分隔的形式保存到名为 result.txt 的文件中。
提示：

请使用 NumPy 库来处理数组和矩阵。
考虑对非规范氨基酸的处理，确保程序能够正确处理各种输入情况。
可以添加命令行参数来接收输入文件名、输出文件名以及其他必要的参数。 -->

In [ ]:
import sys
import numpy as np
import pandas as pd  
def makeAA531Dict(AA531FileName):
    filename = AA531FileName
    df = pd.read_csv(filename, delimiter='\t', header=None, skiprows=1)
    AA531Dict = df.set_index(0).T.to_dict('list')#选取第一列作为索引，‘.T’转置，‘to_dict('list')’将其转换为字典。其中字典的值是一个列表，包含每一列的值
    return AA531Dict

In [ ]:
def file2matrix(AASeqFileName,seqlength,AA531Dict):
    filename = AASeqFileName
    df = pd.read_csv(AASeqFileName,sep="\t",header=None)
    #得到行数numberOFlines，开辟内存数组returnMat，同时准备一个二维数组储存三肽数字标签Y
    numberOFlines = len(df)
    seqlength = seqlength
    returnMat = np.zeros((numberOFlines,seqlength*531))
    Y = np.zeros((numberOFlines,1))
    #遍历每个三肽，与字典比对，添加到一个新的列表中
    for lineNum in range(numberOFlines):
        Y[lineNum] = float(df.iloc[lineNum,1])
        feaVec = []
        for AA in df.iloc[lineNum,0]:
            if AA in AA531Dict.keys():
                feaVec.extend(AA531Dict[AA])
            else:
                print('Warning: nonregular amino acid found! Coding "%s" in "%s" (seqId: %d) with 531 zeros.' % (AA, AAseq, lineNum))
                feaVec.extend([0.0]*531)
                Y[lineNum] = -1
        returnMat[lineNum,:] = np.array(feaVec)
        lineNum += 1
    return Y,returnMat,lineNum
if __name__ == '__main__':
   AASeqFileName = sys.argv[1]
   AA531FileName = sys.argv[2]
   seqLength = int(sys.argv[3])
   outputFileName = sys.argv[4]
   AA531Dict = makeAA531Dict(AA531FileName)
   Y, AA531Mat, SeqNum = file2matrix(AASeqFileName, seqLength, AA531Dict)
   np.savetxt(outputFileName, np.hstack((Y, AA531Mat)), fmt='%g', delimiter='\t')
   print('The number of sequences is %d. Matrix of features is saved in %s' % (SeqNum, outputFileName))